In [48]:
import random

# total number of cylinders
tot_cylinder = 200
# current head position
curr_head = 50
# moving direction of the disk head
moving_direction = "right"

In [49]:
# SCAN ALGORITHM

def SCAN(requests, curr_head, moving_direction):

    # initialising all the variables
    # the total seek time to process every request
    seek_time = 0
    
    # the distance from the current disk head to the next nearest requested track
    # curr_track is to store the next nearest requested track
    track_traveled, curr_track = 0, 0
    
    # store requested tracks on the left-hand side of the current disk head
    left_request = []
    
    # store requested tracks on the right-hand side of the current disk head
    right_request = []
    
    # the requested tracks sequence visited by the disk head,
    # includes the first/last track, depending on initial direction
    seek_req_sequence = []

    # append the end value that the disk head will visit before reversing the direction
    if moving_direction == "left":
        left_request.append(0)
    elif moving_direction == "right":
        right_request.append(tot_cylinder - 1)

    # append the requested tracks to store on the left or right-hand side of the current disk head
    # if a request 50 is in the requested tracks, then put it on the left requests.
    for i in range(len(requests)):
        if moving_direction == "left":
            if requests[i] <= curr_head:
                left_request.append(requests[i])
            else:
                right_request.append(requests[i])
        elif moving_direction == "right":
            if requests[i] < curr_head:
                left_request.append(requests[i])
            else:
                right_request.append(requests[i])

    # Sorting in descending order for left requests
    # Sorting in ascending order for right requests
    left_request.sort(reverse=True)
    right_request.sort()
    
    # Display the left and right requests
    print("\nSCAN:")
    print("Left Requests (SCAN) =", left_request)
    print("Right Requests (SCAN) =", right_request)

    # Will loop two times
    # One to the left and another one to the right of the disk head, and vice versa
    loop = 2
    while loop != 0:
        if moving_direction == "left":
            for i in range(len(left_request)):
                curr_track = left_request[i]

                # append the current track to the seek request sequence
                seek_req_sequence.append(curr_track)

                # calculate the absolute distance traveled by the disk head
                track_traveled = abs(curr_track - curr_head)

                # increase the seek time
                seek_time += track_traveled

                # visited track becomes the new disk head
                curr_head = curr_track

            # reverse the direction
            moving_direction = "right"
            

        elif moving_direction == "right":
            for i in range(len(right_request)):
                curr_track = right_request[i]

                # append the current track to the seek request sequence
                seek_req_sequence.append(curr_track)

                # calculate the absolute distance traveled by the disk head
                track_traveled = abs(curr_track - curr_head)

                # increase the seek time
                seek_time += track_traveled

                # visited track becomes the new disk head
                curr_head = curr_track

            # reverse the direction
            moving_direction = "left"


        loop -= 1
        
        
    # Display Seek Request Sequence, which includes 0 or 199, depending on initial direction
    print("Seek request sequence =", seek_req_sequence)
    
    # Display total seek time
    print("Total seek time =", seek_time, "ms")

    # Calculate average seek time
    num_requests = len(requests)
    average_seek_time = seek_time / num_requests
    print("Average seek time =", average_seek_time, "ms")
    
    # Calculate worst-case seek time
    # Identify max seek time between two requests, which is always when final request of one direction to first request of
    # the other direction.
    
    if moving_direction == "left":
        # Find the 2nd last request in left_request
        req1 = left_request[-2]

        # Find the first request in right_request
        req2 = right_request[0]

        # Calculate the maximum distance
        max_distance = ((req1 - 0) + (req2 - 0))

        print("Worst-case seek time =", max_distance, "ms")
        
    elif moving_direction == "right":
        # Find the 2nd last request in right_request
        req1 = right_request[-2]

        # Find the first request in left_request
        req2 = left_request[0]

        # Calculate the maximum distance
        max_distance = (((tot_cylinder - 1) - req1) + ((tot_cylinder - 1) - req2))

        print("Worst-case seek time =", max_distance, "ms")
      

In [50]:
# C-SCAN ALGORITHM

def CSCAN(requests, curr_head, moving_direction):
    # Initialising all the variables
    seek_time = 0
    track_travelled = 0
    curr_track = 0
    seek_req_sequence = []
    left_request = []
    right_request = []
    
    # append the end value that the disk head will visit before reversing the direction
    left_request.append(0)
    right_request.append(tot_cylinder - 1)
    
    # Sort the request list in ascending order
    requests.sort()
    
    # Find the index where the current head is located in the sorted request list
    curr_head_index = 0
    for i in range(len(requests)):
        if requests[i] >= curr_head:
            curr_head_index = i
            break

    # Calculate left and right requests for C-SCAN
    for i in range(curr_head_index, len(requests)):
        right_request.append(requests[i])

    for i in range(0, curr_head_index):
        left_request.append(requests[i])


    # Sorting in descending order for left and ascending order for right requests
    left_request.sort(reverse=True)
    right_request.sort()

    # Display the left and right requests
    print("\nC-SCAN:")
    print("Left Requests (C-SCAN) =", left_request)
    print("Right Requests (C-SCAN) =", right_request)

    if moving_direction == "right":
    # First service the requests on the right side of the head.
        for i in range(curr_head_index, len(requests)):
            curr_track = requests[i]

            # Appending current track to seek sequence
            seek_req_sequence.append(curr_track)

            # Calculate absolute distance travelled by the disk head
            track_travelled = abs(curr_track - curr_head)

            # Increase the seek time
            seek_time += track_travelled

            # Accessed track becoming the new head
            curr_head = curr_track
        
        # The C-SCAN algorithm will visit the end value before reversing the direction
        seek_req_sequence.append(tot_cylinder -1)
        
        # Increase the seek time from the last request to the end value
        seek_time += ((tot_cylinder -1) - curr_head)
        
        # Once reached the right end jump to the beginning.
        curr_head = 0
        seek_req_sequence.append(0)
        
        # adding seek time for head returning to 0
        seek_time += (tot_cylinder -1)

        # Now service the requests again which are left.
        for i in range(0, curr_head_index):
            curr_track = requests[i]
            seek_req_sequence.append(curr_track)
            track_travelled = abs(curr_track - curr_head)
            seek_time += track_travelled
            curr_head = curr_track

    # First service the requests on the left side of the head.
    elif moving_direction == "left":
        for i in range(curr_head_index - 1, -1, -1):
            curr_track = requests[i]
            seek_req_sequence.append(curr_track)
            track_travelled = abs(curr_track - curr_head)
            seek_time += track_travelled
            curr_head = curr_track
        
        # The C-SCAN algorithm will visit the end value before reversing the direction
        seek_req_sequence.append(0)
        
        # Increase the seek time from the last request to the end value
        seek_time += curr_head
        
        # Once reached the left end jump to the right end.
        curr_head = tot_cylinder -1
        seek_req_sequence.append(tot_cylinder -1)
        
        # adding seek time from left end to right end
        seek_time += (tot_cylinder -1)

        # Now service the requests again which are right.
        for i in range(len(requests) - 1, curr_head_index - 1, -1):
            curr_track = requests[i]
            seek_req_sequence.append(curr_track)
            track_travelled = abs(curr_track - curr_head)
            seek_time += track_travelled
            curr_head = curr_track
    

    # Print seek request sequence
    print("Seek request sequence =", seek_req_sequence)
    
    # Print all the results, starting with total seek time
    print("Total seek time =", seek_time, "ms")

    # Calculate average seek times for C-SCAN
    num_requests = len(requests)
    average_seek_time = seek_time / num_requests
    print("Average seek time =", average_seek_time, "ms")

    # Calculate worst-case seek times for C-SCAN
    if moving_direction == "left":
        # Find the 2nd last request in left_request
        req1 = left_request[-2]
        
        req2 = right_request[-2]
        # Calculate the maximum distance
        max_distance = ((req1 - 0) + (tot_cylinder - 1) + (tot_cylinder - 1 - req2))

        print("Worst-case seek time =", max_distance, "ms")

    elif moving_direction == "right":
        # Find the 2nd last request in right_request
        req1 = right_request[-2]
        
        req2 = left_request[-2]

        # Calculate the maximum distance
        max_distance = ((tot_cylinder - 1 - req1) + (tot_cylinder - 1) + (req2 - 0))

        print("Worst-case seek time =", max_distance, "ms")

In [51]:
# C-LOOK ALGORITHM

def CLOOK(requests, curr_head, moving_direction):
    # Initialize variables
    seek_time = 0
    seek_req_sequence = []
    left_request = []
    right_request = []

    # Arrange the list of requests in ascending order.
    requests.sort()

    # Locate the index in the sorted request list that corresponds to the current head.
    curr_head_index = 0
    for i in range(len(requests)):
        if requests[i] >= curr_head:
            curr_head_index = i
            break
            
    # Calculate left and right requests for C-LOOK
    for i in range(curr_head_index, len(requests)):
        right_request.append(requests[i])

    for i in range(0, curr_head_index):
        left_request.append(requests[i])


    # Sorting in descending order for left and ascending order for right requests
    left_request.sort(reverse=True)
    right_request.sort()

    # Display the left and right requests
    print("\nC-LOOK:")
    print("Left Requests (C-LOOK) =", left_request)
    print("Right Requests (C-LOOK) =", right_request)

    if moving_direction == "right":
        # Process requests from the current head position to the end of the disk
        for i in range(curr_head_index, len(requests)):
            track = requests[i]
            seek_req_sequence.append(track)
            seek_time += abs(track - curr_head)
            curr_head = track

        # Process requests from the beginning of the disk to the current head position
        for i in range(0, curr_head_index):
            track = requests[i]
            seek_req_sequence.append(track)
            seek_time += abs(track - curr_head)
            curr_head = track
    elif moving_direction == "left":
        # Process requests from the current head position to the beginning of the disk
        for i in range(curr_head_index - 1, -1, -1):
            track = requests[i]
            seek_req_sequence.append(track)
            seek_time += abs(track - curr_head)
            curr_head = track

        # Process requests from the end of the disk to the current head position
        for i in range(len(requests) - 1, curr_head_index - 1, -1):
            track = requests[i]
            seek_req_sequence.append(track)
            seek_time += abs(track - curr_head)
            curr_head = track

    # Calculate average seek time
    num_requests = len(requests)
    average_seek_time = seek_time / num_requests

    # Calculate worst-case seek time
    # The maximum distance between any track in the seek request sequence seek_req_sequence 
    # and the starting head position (50) is determined by this code.
    max_distance = max(abs(seek_req_sequence[0] - 50), max(abs(track - seek_req_sequence[index]) for index, 
                                                           track in enumerate(seek_req_sequence[1:])))
  
    # Print results
    print("Seek request sequence is:", seek_req_sequence)
    print("Total seek time =", seek_time, "ms")
    print("Average seek time =", average_seek_time, "ms")
    print("Worst-case seek time =", max_distance, "ms")


In [52]:
# generate a set of 10 random requests
# request_10 = [141, 159, 86, 173, 17, 13, 41, 96, 147, 144]
request_10 = random.sample(range(tot_cylinder), 10)
print("\nRequest size : 10")
print("10 random requests :", request_10)
SCAN(request_10, curr_head, moving_direction)
CSCAN(request_10, curr_head, moving_direction)
CLOOK(request_10, curr_head, moving_direction)


Request size : 10
10 random requests : [141, 159, 86, 173, 17, 13, 41, 96, 147, 144]

SCAN:
Left Requests (SCAN) = [41, 17, 13]
Right Requests (SCAN) = [86, 96, 141, 144, 147, 159, 173, 199]
Seek request sequence = [86, 96, 141, 144, 147, 159, 173, 199, 41, 17, 13]
Total seek time = 335 ms
Average seek time = 33.5 ms
Worst-case seek time = 184 ms

C-SCAN:
Left Requests (C-SCAN) = [41, 17, 13, 0]
Right Requests (C-SCAN) = [86, 96, 141, 144, 147, 159, 173, 199]
Seek request sequence = [86, 96, 141, 144, 147, 159, 173, 199, 0, 13, 17, 41]
Total seek time = 389 ms
Average seek time = 38.9 ms
Worst-case seek time = 238 ms

C-LOOK:
Left Requests (C-LOOK) = [41, 17, 13]
Right Requests (C-LOOK) = [86, 96, 141, 144, 147, 159, 173]
Seek request sequence is: [86, 96, 141, 144, 147, 159, 173, 13, 17, 41]
Total seek time = 311 ms
Average seek time = 31.1 ms
Worst-case seek time = 160 ms


In [56]:
# generate a set of 20 random requests
request_20 = random.sample(range(tot_cylinder), 20)
print("\nRequest size : 20")
print("20 random requests :", request_20)
SCAN(request_20, curr_head, moving_direction)
CSCAN(request_20, curr_head, moving_direction)
CLOOK(request_20, curr_head, moving_direction)


Request size : 20
20 random requests : [14, 198, 60, 190, 81, 32, 175, 113, 107, 89, 65, 75, 119, 30, 160, 128, 56, 191, 108, 39]

SCAN:
Left Requests (SCAN) = [39, 32, 30, 14]
Right Requests (SCAN) = [56, 60, 65, 75, 81, 89, 107, 108, 113, 119, 128, 160, 175, 190, 191, 198, 199]
Seek request sequence = [56, 60, 65, 75, 81, 89, 107, 108, 113, 119, 128, 160, 175, 190, 191, 198, 199, 39, 32, 30, 14]
Total seek time = 334 ms
Average seek time = 16.7 ms
Worst-case seek time = 161 ms

C-SCAN:
Left Requests (C-SCAN) = [39, 32, 30, 14, 0]
Right Requests (C-SCAN) = [56, 60, 65, 75, 81, 89, 107, 108, 113, 119, 128, 160, 175, 190, 191, 198, 199]
Seek request sequence = [56, 60, 65, 75, 81, 89, 107, 108, 113, 119, 128, 160, 175, 190, 191, 198, 199, 0, 14, 30, 32, 39]
Total seek time = 387 ms
Average seek time = 19.35 ms
Worst-case seek time = 214 ms

C-LOOK:
Left Requests (C-LOOK) = [39, 32, 30, 14]
Right Requests (C-LOOK) = [56, 60, 65, 75, 81, 89, 107, 108, 113, 119, 128, 160, 175, 190, 191, 1

In [57]:
# generate a set of 50 random requests
request_50 = random.sample(range(tot_cylinder), 50)
print("\nRequest size : 50")
print("50 random requests :", request_50)
SCAN(request_50, curr_head, moving_direction)
CSCAN(request_50, curr_head, moving_direction)
CLOOK(request_50, curr_head, moving_direction)


Request size : 50
50 random requests : [56, 67, 155, 60, 94, 122, 158, 16, 63, 117, 190, 43, 49, 90, 115, 28, 188, 37, 164, 141, 8, 23, 38, 96, 55, 54, 27, 65, 75, 154, 147, 124, 85, 125, 151, 53, 161, 150, 84, 180, 61, 116, 120, 184, 30, 111, 48, 4, 171, 22]

SCAN:
Left Requests (SCAN) = [49, 48, 43, 38, 37, 30, 28, 27, 23, 22, 16, 8, 4]
Right Requests (SCAN) = [53, 54, 55, 56, 60, 61, 63, 65, 67, 75, 84, 85, 90, 94, 96, 111, 115, 116, 117, 120, 122, 124, 125, 141, 147, 150, 151, 154, 155, 158, 161, 164, 171, 180, 184, 188, 190, 199]
Seek request sequence = [53, 54, 55, 56, 60, 61, 63, 65, 67, 75, 84, 85, 90, 94, 96, 111, 115, 116, 117, 120, 122, 124, 125, 141, 147, 150, 151, 154, 155, 158, 161, 164, 171, 180, 184, 188, 190, 199, 49, 48, 43, 38, 37, 30, 28, 27, 23, 22, 16, 8, 4]
Total seek time = 344 ms
Average seek time = 6.88 ms
Worst-case seek time = 159 ms

C-SCAN:
Left Requests (C-SCAN) = [49, 48, 43, 38, 37, 30, 28, 27, 23, 22, 16, 8, 4, 0]
Right Requests (C-SCAN) = [53, 54, 55

In [67]:
# generate a set of 100 random requests
request_100 = random.sample(range(tot_cylinder), 100)
print("\nRequest size : 100")
print("100 random requests :", request_100)
SCAN(request_100, curr_head, moving_direction)
CSCAN(request_100, curr_head, moving_direction)
CLOOK(request_100, curr_head, moving_direction)


Request size : 100
100 random requests : [57, 121, 174, 180, 73, 186, 108, 161, 176, 98, 58, 48, 181, 70, 104, 139, 35, 147, 17, 42, 89, 27, 124, 138, 148, 76, 54, 137, 193, 12, 99, 153, 68, 63, 150, 23, 34, 20, 113, 30, 131, 49, 136, 151, 123, 175, 129, 87, 103, 4, 11, 22, 189, 37, 90, 187, 156, 66, 9, 16, 36, 159, 196, 10, 41, 93, 135, 81, 67, 40, 169, 173, 33, 155, 100, 94, 101, 140, 88, 182, 1, 111, 105, 130, 162, 43, 133, 197, 106, 110, 13, 183, 96, 44, 62, 179, 166, 74, 45, 61]

SCAN:
Left Requests (SCAN) = [49, 48, 45, 44, 43, 42, 41, 40, 37, 36, 35, 34, 33, 30, 27, 23, 22, 20, 17, 16, 13, 12, 11, 10, 9, 4, 1]
Right Requests (SCAN) = [54, 57, 58, 61, 62, 63, 66, 67, 68, 70, 73, 74, 76, 81, 87, 88, 89, 90, 93, 94, 96, 98, 99, 100, 101, 103, 104, 105, 106, 108, 110, 111, 113, 121, 123, 124, 129, 130, 131, 133, 135, 136, 137, 138, 139, 140, 147, 148, 150, 151, 153, 155, 156, 159, 161, 162, 166, 169, 173, 174, 175, 176, 179, 180, 181, 182, 183, 186, 187, 189, 193, 196, 197, 199]
Se

Static Requests for Analysis Purposes:

Request size : 10

10 random requests : [141, 159, 86, 173, 17, 13, 41, 96, 147, 144]


Request size : 20

20 random requests : [14, 198, 60, 190, 81, 32, 175, 113, 107, 89, 65, 75, 119, 30, 160, 128, 56, 191, 108, 39]


Request size : 50

50 random requests : [56, 67, 155, 60, 94, 122, 158, 16, 63, 117, 190, 43, 49, 90, 115, 28, 188, 37, 164, 141, 8, 23, 38, 96, 55, 54, 27, 65, 75, 154, 147, 124, 85, 125, 151, 53, 161, 150, 84, 180, 61, 116, 120, 184, 30, 111, 48, 4, 171, 22]


Request size : 100

100 random requests : [57, 121, 174, 180, 73, 186, 108, 161, 176, 98, 58, 48, 181, 70, 104, 139, 35, 147, 17, 42, 89, 27, 124, 138, 148, 76, 54, 137, 193, 12, 99, 153, 68, 63, 150, 23, 34, 20, 113, 30, 131, 49, 136, 151, 123, 175, 129, 87, 103, 4, 11, 22, 189, 37, 90, 187, 156, 66, 9, 16, 36, 159, 196, 10, 41, 93, 135, 81, 67, 40, 169, 173, 33, 155, 100, 94, 101, 140, 88, 182, 1, 111, 105, 130, 162, 43, 133, 197, 106, 110, 13, 183, 96, 44, 62, 179, 166, 74, 45, 61]